<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/week4/Building_your_Deep_Neural_Network_Step_by_Step_v8a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 단계별로 심층 신경망을 만들어봅시다! #

4주차 첫 번째 과제에 오신것을 환영합니다! 이전 주차 과제에서 여러분들은 2개의 레이어, 특히 1개의 은닉층을 가지는 신경망을 만들어 보았습니다. 이번주에는, 레이어가 꽤 많은 심층 신경망을 만들어 볼 것입니다.

- 이 주피터 노트북에서, 여러분들은 심층 신경망을 만들기 위해 필요한 여러가지 함수들을 구현할 것입니다.
- 이후 4주차 두 번째 과제에서는, 이 함수들을 사용헤서 이미지 분류를 할 수 있는 심층 신경망을 만들 것입니다.

**이 과제가 끝나고 나면:**
- 비선형 ReLU 함수를 사용해 모델 성능을 향상시킬 수 있습니다.
- 1개 이상의 은닉층을 사용하는 심층 신경망을 만들 수 있습니다.
- 쉽게 재사용할 수 있는 신경망 클래스를 개발할 수 있습니다.

**표기법(Notation):**
- 위첨자 $[l]$ 은 해당 변수가 $l$번째 층과 연관된 변수라는 것을 의미합니다.
  - $a^{[L]}$ 은 $L$ 번째 층의 활성화 값을 의미합니다.
  - $W^{[L]}$ 과 $b^{[L]}$ 변수는 $L$ 번째 층의 파라미터를 의미합니다.
- 위첨자 $(i)$ 는 해당 변수가 데이터 셋의 $i$ 번째 데이터와 연관이 있다는 것을 의미합니다.
  - $x^{(i)}$ 는 $i$ 번째 훈련 데이터를 의미합니다
- 아랫첨자 $i$ 는 벡터의 $i$ 번째 성분을 의미합니다.
  -  $a^{[l]}_i$ 는 $l$ 번째 층의 활성화 값들 중 $i$ 번째 값을 의미합니다.).

이제 시작해 봅시다!

## 1. 패키지 다운로드하기

늘 그랬듯이, 가장 먼저 이 과제를 수행하는 동안 필요한 다양한 패키지들을 다운로드해봅시다.

- [numpy](www.numpy.org)는 파이썬의 가장 기본적인 과학/수학 연산 관련 패키지입니다.
- [matplotlib](http://matplotlib.org)은 파이썬 환경에서 그래프를 그릴 수 있도록 해주는 유명한 라이브러리입니다.
- `dnn_utils` 는 과제를 수행하기 위해 필요한 다양한 빌트인 함수들을 제공합니다.
- `testCases` 는 과제에서 구현한 함수가 잘 작동하는지 테스트할 수 있는 예시들을 제공합니다.
- `np.random.seed(1)` 은 랜덤 함수가 일관성을 유지하도록 시드를 설정해줍니다. 과제를 채점하기 위해 필요한 사항이므로, 강제로 시드를 변경하지 마세요.

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v4a import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

## 2. 전반적인 과제의 개요 ##

심층 신경망을 만들기 위해, 지금부터 몇가지 "helper" 함수들을 만들도록 하겠습니다. 이 helper 함수들은 다음 과제에서 2-layer와 L-layer인 심층 신경망을 만드는데 사용될 것입니다. 직접 helper 함수들을 구현할 때 각 단계를 안내하는 자세한 지시사항이 있습니다.

결론적으로, 이 과제의 개요는 다음과 같습니다:
- 2-layer 신경망과 $L$-layer 신경망의 파라미터를 초기화합니다.
- `forward propagation`를 담당하는 모듈을 구현합니다(아래 그림에서 보라색에 해당합니다)
  - forward propagation 단계 중 선형 부분 (($Z^{[l]}$ 을 계산하는 부분) 을 완성합니다.
  - 활성화 함수 (ReLU, sigmoid)를 구현합니다.
  - 두 단계를 합쳐 [선형 부분 -> 활성화 함수] 로 이어지는 forward 함수를 구현합니다.
  - [LINEAR->RELU] `forward` 함수를 $L-1$ 만큼 쌓고, (1번째 층부터 L-1번째 층 까지) 이후 [LINEAR->SIGMOID] `forward` 함수를 마지막 층 $L$에 추가하여 L_model `forward` 함수를 완성합니다.
- `loss`를 계산합니다.
- `backward propagation` 을 구현합니다(아래 그림에서 빨간 색에 해당합니다)
  - 오차 역전파 과정 중 선형 부분을 구현합니다.
  - 활성화 함수의 `gradient`를 계산합니다. (`relu_backward` / `sigmoid_backward`)
  - 두 단계를 합쳐 새 [LINEAR->ACTIVATION] `backward` 함수를 만듭니다.
  - [LINEAR->RELU] `backward` 함수를 $L-1$ 번 만큼 쌓고, [LINEAR->SIGMOID] `backward` 함수를 끝에 추가하여 L_model `backward` 함수를 완성합니다.
- 최종적으로 파라미터를 업데이트합니다.

<img src="arts/final outline.png" style="width:800px;height:500px;">
<caption><center>그림 1</center></caption></br>

**매 forward 함수에 대해 상응하는 backward 함수가 있다는 것을 기억**하세요. 이 때문이 우리는 forward 모듈을 동작시킬 때마다 몇 가지 변수들을 캐시에 저장해야 합니다. 이렇게 저장된 캐시 변수는 이후 gradient를 계산할 때 사용될 수 있습니다. 다시 말해 backpropagation 모듈이 동작할 때 이 캐시 데이터를 활용해서 gradient를 계산합니다. 이 과제에선 일련의 단계를 어떻게 수행하는지 보여줄 예정입니다.


## 3. 초기화 ##

이제 인공 신경망의 파라미터를 초기화하는 두 가지 helper 함수를 만들어봅시다. 첫 번째 함수는 2개의 층을 가지는 모델의 파라미터를 초기화하는데 사용될 것이고, 두 번째는 L개의 층을 가지는 모델을 위해 사용됩니다.

### 3-1. 2-layer 신경망 ###

**연습 문제** : 2개 층을 가지는 인공 신경망의 파라미터를 초기화 해봅시다.

**지시 사항** :
- 모델의 구조는 선형 함수 -> ReLU -> 선형 함수 -> sigmoid 입니다.
- 가중치 행렬을 랜덤하게 초기화하세요. 올바른 shape에 대해 `np.random.randn(shape) * 0.01` 코드를 사용해 보세요.
- bias는 0으로 초기화합니다. `np.zeros(shape)` 를 사용해보세요.

In [ ]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    parameters -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(1)
    
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = None
    b1 = None
    W2 = None
    b2 = None
    ### END CODE HERE ###
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters 

In [ ]:
parameters = initialize_parameters(3,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Expected output**:
<table style="width:80%">
  <tr>
    <td>W1</td>
    <td> [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]] </td> 
  </tr>
  <tr>
    <td> b1</td>
    <td>[[ 0.]
 [ 0.]]</td> 
  </tr>
  <tr>
    <td>W2</td>
    <td> [[ 0.01744812 -0.00761207]]</td>
  </tr>
  <tr>
    <td> b2 </td>
    <td> [[ 0.]] </td> 
  </tr>
</table>

### 3-2. L-layer 신경망 ###

L개의 층을 가지는 신경망은 많은 가중치 행렬과 bias 벡터를 초기화해야 하기 때문에 초기화 과정이 꽤 어렵습니다. 아래에서 `initialize_parameters_deep()` 함수를 구현하면서, 각 층의 차원이 정확한지 확인해야 합니다. $n^{[l]}$ 값이 각 층에 있는 유닛의 개수라는 것을 기억하세요. 따라서 만약 샘플 데이터인 입력 행렬 X가 $(12288, 209)$, (개수 $m=209$) 라고 한다면, 각각 층의 가중치는 다음과 같을 것입니다.

</br>

<table style="width:100%">
    <tr>
        <td></td> 
        <td> Shape of W </td> 
        <td> Shape of b  </td> 
        <td> Activation </td>
        <td> Shape of Activation </td> 
    <tr>
    <tr>
        <td> Layer 1 </td> 
        <td> $(n^{[1]},12288)$ </td> 
        <td> $(n^{[1]},1)$ </td> 
        <td> $Z^{[1]} = W^{[1]}  X + b^{[1]} $ </td>    
        <td> $(n^{[1]},209)$ </td> 
    <tr>
    <tr>
        <td> Layer 2 </td> 
        <td> $(n^{[2]}, n^{[1]})$  </td> 
        <td> $(n^{[2]},1)$ </td> 
        <td>$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$ </td> 
        <td> $(n^{[2]}, 209)$ </td> 
    <tr>
       <tr>
        <td> $\vdots$ </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$</td> 
        <td> $\vdots$  </td> 
    <tr>
   <tr>
        <td> Layer L-1 </td> 
        <td> $(n^{[L-1]}, n^{[L-2]})$ </td> 
        <td> $(n^{[L-1]}, 1)$  </td> 
        <td>$Z^{[L-1]} =  W^{[L-1]} A^{[L-2]} + b^{[L-1]}$ </td> 
        <td> $(n^{[L-1]}, 209)$ </td> 
    <tr>
   <tr>
        <td> Layer L </td> 
        <td> $(n^{[L]}, n^{[L-1]})$ </td> 
        <td> $(n^{[L]}, 1)$ </td>
        <td> $Z^{[L]} =  W^{[L]} A^{[L-1]} + b^{[L]}$</td>
        <td> $(n^{[L]}, 209)$  </td> 
    <tr>
</table>

</br>

$WX + b$ 공식을 계산할 때 파이썬에서는 broadcasting 이 적용된다는 사실을 잊지 마세요.

$$ W = \begin{bmatrix}
    j  & k  & l\\
    m  & n & o \\
    p  & q & r 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    a  & b  & c\\
    d  & e & f \\
    g  & h & i 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    s  \\
    t  \\
    u
\end{bmatrix}\tag{2}$$

위와 같은 상황에서 $WX + b$ 는 아래와 같이 계산될 것입니다.

$$ WX + b = \begin{bmatrix}
    (ja + kd + lg) + s  & (jb + ke + lh) + s  & (jc + kf + li)+ s\\
    (ma + nd + og) + t & (mb + ne + oh) + t & (mc + nf + oi) + t\\
    (pa + qd + rg) + u & (pb + qe + rh) + u & (pc + qf + ri)+ u
\end{bmatrix}\tag{3}  $$



**연습 문제**: L-layer 신경망을 초기화하는 함수를 구현하세요.

**지시 사항**:
- 모델 구조: `[선형 함수 -> ReLU]` L-1회 반복 -> 선형 함수 -> `sigmoid`
- 가중치 행렬을 랜덤하게 초기화하세요. 올바른 shape에 대해 `np.random.randn(shape) * 0.01` 코드를 사용해 보세요.
- bias는 0으로 초기화합니다. `np.zeros(shape)` 를 사용해보세요.
- 각 층의 unit들의 개수 $n^{[l]}$ 가 `layer_dims` 변수에 저장됩니다. 
  - 예를 들어, 지난 주의 "평면 데이터 분류 모델" 에 대한 `layer_dims`는 [2, 4, 1] 이 됩니다. 해석하자면 입력 층에는 두 개의 unit, 첫 번째 은닉 층은 4개의 unit, 출력 유닛 1개로 구성되어 있다는 것을 의미합니다.
  - 이는 W1의 `shape`는 (4,1), b1이 (4,1) W2가 (1,4), b2가 (1,1)임을 의미합니다. 이제 이 값들을 L개의 층을 가진 모델로 일반화합니다.
- 다음은 $L = 1$(단일층 신경망) 에 대한 구현입니다. 이 예시는 L 개의 층을 가진 신경망을 구현하는데 도움을 줄 것입니다.
  ```python
      if L == 1:
          parameters["W" + str(L)] = np.random.randn(layer_dims[1], layer_dims[0]) * 0.01
          parameters["b" + str(L)] = np.zeros((layer_dims[1], 1))
  ```

In [ ]:
# GRADED FUNCTION: initialize_parameters_deep

def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = None
        parameters['b' + str(l)] = None
        ### END CODE HERE ###
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [ ]:
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Expected output**:
       
<table style="width:80%">
  <tr>
    <td> W1 </td>
    <td>[[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]</td> 
  </tr>
  <tr>
    <td> b1 </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]</td> 
  </tr>
  <tr>
    <td> W2 </td>
    <td>[[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]</td> 
  </tr>
  <tr>
    <td> b2 </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]]</td> 
  </tr>
</table>

## 4. Forward Propagation 모듈 구현하기 ##


### 4-1. Linear Forward 계산하기 ##

이제 초기화된 파라미터를 가지고, forward propagation 모듈을 만들어봅시다. 모델을 구현하기 위한 기본적인 기능을 담당하는 함수를 만드는 것으로 시작해봅시다. 세 가지 함수를 아래와 같은 순서로 완성하면 됩니다.

- 선형 함수 (LINEAR)
- ReLU / sigmoid 의 활성화 함수 (LINEAR -> ACTIVATION)
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID (전체 모델)

모든 테스트 데이터에 대하여 벡터화된 선형 forward 모듈은 아래 공식을 따라 계산합니다.

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$

추가로, $A^{[0]} = X$ 입니다. 

**연습 문제**: forward propagation 함수의 선형 부분을 구현해보세요.

**복습** : 이 부분의 수학적 표현은 $Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$ 입니다. 이전과 마찬가지로 `np.dot()` 함수를 쓰는 것이 유용할 것입니다. 만약 행렬의 차원을 맞추지 못하겠다면, `W.shape`를 사용해보세요.

In [ ]:
# GRADED FUNCTION: linear_forward

def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python tuple containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    ### START CODE HERE ### (≈ 1 line of code)
    Z = None
    ### END CODE HERE ###
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [ ]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

**모범 답안**:

<table style="width:35%">
  
  <tr>
    <td> Z </td>
    <td> [[ 3.26295337 -1.23429987]] </td> 
  </tr>
  
</table>

### 4-2. 선형 활성화 함수 구현하기 ###

이번 과제에서 여러분은 두 가지 활성화 함수를 사용할 수 있습니다.

- **Sigmoid** :  $\sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}}$. 이번 과제에서는 사전에 구현된 `sigmoid()` 함수를 제공합니다. 이 함수는 **두 개의** 변수를 리턴합니다.
  - 활성화 값 `A`
  - 이 정방향 함수와 상응하는 역방향 함수에서 사용할 "Z" 값을 담고 있는 `cache` 변수
  - sigmoid 함수는 아래 코드처럼 사용하세요.
    ```python
    A, activation_cache = sigmoid(Z)
    ```

- **ReLU**: ReLU 함수의 수학적인 공식은 $A = RELU(Z) = max(0, Z)$ 입니다. 이번 과제에서는 사전에 구현되어 있는 `relu()` 함수를 사용합니다. 이 함수 역시 **두 개의** 변수를 리턴합니다.
  - 활성화 값 `A`
  - 역방향 전파에 사용되는 Z값을 담은 `cache` 변수
  - ReLU는 아래 코드처럼 사용하세요
    ```python
    A, activation_cache = relu(Z)
    ```

</br>

편의를 위해 Linear와 Activation 두 가지 기능을 하나로 합칩니다. 따라서 Linear을 먼저 수행한 다음, Activation을 이어서 수행하는 함수를 구현합니다.

**연습 문제** : Linear -> Activation 을 수행하는 forward propagation를 구현하세요. 
- 활성화 함수 `g`는 `sigmoid()` 또는 `relu()`가 될 수 있습니다. 
- 공식은 $A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} +b^{[l]})$ 입니다.

In [ ]:
# GRADED FUNCTION: linear_activation_forward

def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python tuple containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = None
        A, activation_cache = None
        ### END CODE HERE ###
    
    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = None
        A, activation_cache = None
        ### END CODE HERE ###
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [ ]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

**Expected output**:
<table style="width:35%">
  <tr>
    <td> With sigmoid: A </td>
    <td > [[ 0.96890023  0.11013289]]</td> 
  </tr>
  <tr>
    <td> With ReLU: A </td>
    <td > [[ 3.43896131  0.        ]]</td> 
  </tr>
</table>

</br>

**메모** : 딥 러닝에서 "[LINEAR -> ACTIVATION]"의 일련의 계산은 하나의 층으로 간주합니다.

#### d) L-layer 모델 ####

L-layer 신경망을 구현할 때 더 많은 편의를 위해서 이전에 구현했던 `relu()` 함수를 활성화 함수로 사용하는 `linear_activation_foward()` 를 $L-1$ 회 복사한 함수가 필요합니다. 그리고 나서 `sigmoid()` 함수를 활성화 함수로 사용하는 `linear_activation_forward()` 함수를 추가합니다.

<img src="arts/model_architecture_kiank.png" style="width:600px;height:300px;">
<caption>Figure 2: [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID model</caption>

</br>

**연습 문제**: 위 모델의 전체 forward propagation을 구현하세요.

**지시 사항**: 아래 코드에서 `AL`은 $A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A^{[L-1]} + b^{[L]})$ 을 의미합니다. 이 `AL` 값은 `Yhat` 이라고도 불립니다 (i.e. $A^{[L]}$ = $\hat{Y}$)

**팁**:
- 앞서 작성했던 함수들을 사용하세요.
- [LINEAR -> RELU] 층을 L-1회 반복하기 위해서 반복문을 사용하세요.
- 앞서 언급했던 캐싱 처리할 데이터들을 `caches` 변수에 저장하는 것을 잊지 마세요. `c` 변수를 `list`에 추가하기 위해서는 `list.append(c)`를 사용하면 됩니다.

In [ ]:
# GRADED FUNCTION: L_model_forward

def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_activation_forward() (there are L-1 of them, indexed from 0 to L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        ### START CODE HERE ### (≈ 2 lines of code)
        A, cache = None
        None
        ### END CODE HERE ###
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    ### START CODE HERE ### (≈ 2 lines of code)
    AL, cache = None
    None
    ### END CODE HERE ###
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [ ]:
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

**모범 답안**:
<table style="width:50%">
  <tr>
    <td> **AL** </td>
    <td > [[ 0.03921668  0.70498921  0.19734387  0.04728177]]</td> 
  </tr>
  <tr>
    <td> **Length of caches list ** </td>
    <td > 3 </td> 
  </tr>
</table>

</br>

축하드립니다! 이제 훈련 데이터X를 입력 값으로, 예측값을 담고 있는 $A^{[L]}$ 열 벡터를 출력으로 하는 전체 forward propagation 모듈을 구현했습니다. 이 모듈은 이후 역방향 전파에 사용되는 중간값인 `caches` 값을 기록합니다. $A^{[L]}$ 값을 활용해서 예측 결과의 비용(cost)을 계산할 수 있습니다. 

## 5. 비용 함수 계산하기 ##

이제 순방향 propagation을 구현해봅시다. 먼저 모델이 실제로 학습 중인지 확인하기 위해 cost를 계산해야 합니다. 

**연습 문제**: 비용 $J$를 계산하기 위해서 크로스 엔트로피 함수를 계산하세요. 다음 공식을 활용하면 됩니다 : $$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$


In [ ]:
# GRADED FUNCTION: compute_cost

def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    ### START CODE HERE ### (≈ 1 lines of code)
    cost = None
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

In [ ]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

**모범 답안**:
<table>
    <tr>
    <td> cost </td>
    <td> 0.2797765635793422</td> 
    </tr>
</table>

## 6. Backward propagation 모듈 구현하기 ##

위 forward propagation과 마찬가지로 backward propagation을 위한 helper 함수들을 구현합시다. backward propagation은 각 파라미터에 대한 손실 함수의 gradient를 계산하기 위해 사용된다는 점을 기억하세요.

**리마인더**:

<img src="arts/backprop_kiank.png" style="width:650px;height:250px;">
<caption>Figure 3 : Forward and Backward propagation for LINEAR->RELU->LINEAR->SIGMOID</br>The purple blocks represent the forward propagation, and the red blocks represent the backward propagation.</caption>

이제, forward propagation과 마찬가지로 3단계에 걸쳐 backward propagation을 구현해봅시다.
- 선형(LINEAR) 역전파 구현
- ReLU 혹은 sigmoid 활성화 함수의 미분계수를 계산하는 LINEAR -> ACTIVATION 역방향 활성화 함수 구현
- $L-1$ 회의 LINEAR -> RELU -> LINEAR -> SIGMOID 역전파

### 6-1. Linear 역전파 구현하기 ###

$l$ 번째 층에 대하여, linear part는 : $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$ 입니다.

이미 미분계수 $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$ 를 계산했다고 가정하면, 이어서 $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$ 를 구할 수 있습니다.

<img src="arts/linearback_kiank.png" style="width:250px;height:300px;">
<caption>Figure 4</caption>

세 출력 값 $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$ 은 입력값 $dZ^{[l]}$ 을 사용해서 계산할 수 있습니다. 공식은 아래와 같습니다.

$$ dW^{[l]} = \frac{\partial \mathcal{J} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{J} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$

</br>

**연습 문제**: 위의 세 가지 공식을 사용해 `linear_backward()` 함수를 구현하세요.

In [ ]:
# GRADED FUNCTION: linear_backward

def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### START CODE HERE ### (≈ 3 lines of code)
    dW = None
    db = None
    dA_prev = None
    ### END CODE HERE ###
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [ ]:
# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

**모범 답안**:
    
```
dA_prev = 
 [[-1.15171336  0.06718465 -0.3204696   2.09812712]
 [ 0.60345879 -3.72508701  5.81700741 -3.84326836]
 [-0.4319552  -1.30987417  1.72354705  0.05070578]
 [-0.38981415  0.60811244 -1.25938424  1.47191593]
 [-2.52214926  2.67882552 -0.67947465  1.48119548]]
dW = 
 [[ 0.07313866 -0.0976715  -0.87585828  0.73763362  0.00785716]
 [ 0.85508818  0.37530413 -0.59912655  0.71278189 -0.58931808]
 [ 0.97913304 -0.24376494 -0.08839671  0.55151192 -0.10290907]]
db = 
 [[-0.14713786]
 [-0.11313155]
 [-0.13209101]]
```

### 6-2. 선형 활성화 backward 구현하기 ###

다음으로 두 번째 helper 함수 `linear_backward()` 함수를 구현해 봅시다. 

`linear_activation_backward()` 함수를 구현하기 위해서 다음의 두 가지 backwrad 함수를 제공합니다.

- **sigmoid_backward**: `sigmoid()` 활성화 함수의 역방향 도함수입니다. 아래와 같은 방법으로 호출할 수 있습니다.
  ```python
  dZ = sigmoid_backward(dA, activation_cache)
  ```

- **relu_backward**: `relu()` 활성화 함수의 역방향 도함수입니다. 아래와 같은 방법으로 호출할 수 있습니다.
  ```python
  dZ = relu_backward(dA, activation_cache)
  ```

만약 $g(.)$ 이 활성화 함수라면,
`sigmoid_backward`와 `relu_backward`는 $$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) \tag{11}$$
를 계산합니다.

**연습 문제** : *LINEAR -> ACTIVATION 층을 담당하는 역전파 함수를 구현하세요.


In [ ]:
# GRADED FUNCTION: linear_activation_backward

def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        ### START CODE HERE ### (≈ 2 lines of code)
        dZ = None
        dA_prev, dW, db = None
        ### END CODE HERE ###
        
    elif activation == "sigmoid":
        ### START CODE HERE ### (≈ 2 lines of code)
        dZ = None
        dA_prev, dW, db = None
        ### END CODE HERE ###
    
    return dA_prev, dW, db

**sigmoid 함수의 모범 답안:**

<table style="width:100%">
  <tr>
    <td > dA_prev </td> 
           <td >[[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]] </td> 
  </tr> 
    <tr>
    <td > dW </td> 
           <td > [[ 0.10266786  0.09778551 -0.01968084]] </td> 
  </tr> 
    <tr>
    <td > db </td> 
           <td > [[-0.05729622]] </td> 
  </tr> 
</table>

**ReLU 함수의 모범 답안:**

<table style="width:100%">
  <tr>
    <td > dA_prev </td> 
           <td > [[ 0.44090989  0.        ]
 [ 0.37883606  0.        ]
 [-0.2298228   0.        ]] </td> 
  </tr> 
    <tr>
    <td > dW </td> 
           <td > [[ 0.44513824  0.37371418 -0.10478989]] </td> 
  </tr> 
    <tr>
    <td > db </td> 
           <td > [[-0.20837892]] </td> 
  </tr> 
</table>

### 6-3. L-Model Backward 구현 ###

이제 전체 신경망에 대한 역전파 기능을 구현합니다. `L_model_forward()` 함수를 구현할 때 각 반복마다 (X, W, b, z)를 담고 있는 `cache`를 저장했습니다. 역전파 모듈은 이 변수를 사용하여 gradient를 계산합니다. 따라서 `L_model_backward()` 함수는 $L$에서 시작하여 뒤 방향으로 모든 은닉층에 대해 반복적으로 gradient를 계산합니다. 각 단계마다, $l$번 층에서 캐시된 값을 사용하여 역전파를 계산합니다. 아래 그림 5번은 역전파가 어떤 방식으로 이루어지는지 나타내줍니다.

<img src="arts/mn_backward.png" style="width:450px;height:300px;">
<caption>Figure 5 : Backward pass</caption>

</br>

**backpropagation 초기화하기**: 우리는 이 신경망의 출력 $A^{[L]}$이  $\sigma(Z^{[L]})$ 라는 것을 알고 있습니다. 따라서 여러분은 이를 통해서 `dAL` $= \frac{\partial \mathcal{L}}{\partial A^{[L]}}$ 을 계산하는 코드를 작성해야 합니다.
코드를 작성할 때, 미적분에 대한 깊은 지식이 없이도 아래 공식을 사용하여 쉽게 구현할 수 있습니다.
```python
dAL = -(np.divide(Y, AL) - np.divide(1-Y, 1-AL))
```

이렇게 구해진 활성화 값에 대한 gradient `dAL`을 사용해 역전파를 이어서 수행할 수 있습니다. 그림 5번에서 볼 수 있듯이, `dAL` 값을 이전에 구현했던 LINEAR -> SIGMOID backward 함수 (이 함수는 앞서 다루었던 `L_model_forward` 함수에서 캐싱된 데이터를 사용합니다)에 투입할 수 있습니다.

그리고 나서, 반복문을 통해서 다른 모든 층에 대해 LINEAR -> RELU backward 함수를 적용할 수 있습니다. 이 때 각 반복마다 ($dA$, $dW$, $db$) 값을 `grads` 딕셔너리에 저장할 필요가 있습니다. 아래 공식을 사용하세요:

$$grads["dW" + str(l)] = dW^{[l]}\tag{15} $$

예를 들어, $l=3$일 때 이 함수는 $dW^{[l]}$ 값을 `grads["dW3"]`의 형태로 저장할 것입니다.

**연습 문제**: [LINEAR->RELU] $\times$ (L-1) -> LINEAR -> SIGMOID 모델의 backpropagation을 구현하세요.


In [ ]:
# GRADED FUNCTION: L_model_backward

def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    ### START CODE HERE ### (1 line of code)
    dAL = None
    ### END CODE HERE ###
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    ### START CODE HERE ### (approx. 2 lines)
    current_cache = None
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = None
    ### END CODE HERE ###
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        ### START CODE HERE ### (approx. 5 lines)
        current_cache = None
        dA_prev_temp, dW_temp, db_temp = None
        grads["dA" + str(l)] = None
        grads["dW" + str(l + 1)] = None
        grads["db" + str(l + 1)] = None
        ### END CODE HERE ###

    return grads

In [ ]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print_grads(grads)

**모범 답안:**

<table style="width:60%">
  <tr>
    <td > dW1 </td> 
           <td > [[ 0.41010002  0.07807203  0.13798444  0.10502167]
 [ 0.          0.          0.          0.        ]
 [ 0.05283652  0.01005865  0.01777766  0.0135308 ]] </td> 
  </tr> 
    <tr>
    <td > db1 </td> 
           <td > [[-0.22007063]
 [ 0.        ]
 [-0.02835349]] </td> 
  </tr> 
  <tr>
  <td > dA1 </td> 
           <td > [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]] </td> 

  </tr> 
</table>

### 6-4. 파라미터 업데이트 하기 ###

이번에는 경사 하강법을 사용하여 모델의 파라미터를 업데이트하겠습니다.

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{16}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{17}$$

지금까지처럼 $\alpha$ 변수는 학습률입니다. 업데이트된 파라미터를 계산하고 나서 이 값들을 `parameters` 딕셔너리에 집어넣어봅시다.

**연습 문제**: 경사 하강법을 이용하여 파라미터를 업데이트하는 `update_parameters()` 함수를 구현해보세요.

**지시 사항**: 모든 입력 층 $l = 1, 2, ..., L$에 대하여 파라미터 $W^{[l]}$ and $b^{[l]}$ 를 업데이트해야 합니다.

In [ ]:
# GRADED FUNCTION: update_parameters

def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    ### START CODE HERE ### (≈ 3 lines of code)
    for l in range(L):
        parameters["W" + str(l+1)] = None
        parameters["b" + str(l+1)] = None
    ### END CODE HERE ###
    return parameters

In [ ]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

**모범 답안**:

<table style="width:100%"> 
    <tr>
    <td > W1 </td> 
           <td > [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]] </td> 
  </tr> 
    <tr>
    <td > b1 </td> 
           <td > [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]] </td> 
  </tr> 
  <tr>
    <td > W2 </td> 
           <td > [[-0.55569196  0.0354055   1.32964895]]</td> 
  </tr> 
    <tr>
    <td > b2 </td> 
           <td > [[-0.84610769]] </td> 
  </tr> 
</table>

## 7. 결론 ##

심층 신경망을 구현하기 위해 필요한 모든 함수들을 성공적으로 구현했습니다. 꽤나 긴 과제였지만, 그래도 한 시름 놓았습니다. 이후 진행할 과제를 더 쉽게 수행할 수 있을 겁니다. 다음 과제에선 이번에 구현한 모든 함수를 합쳐서 두 모델을 만들 것입니다.
- 2-layer 신경망
- L-layer 신경망

좀 더 구체적으로는, 이 신경망들을 이용해 고양이와 고양이가 아닌 이미지를 분류하는 모델을 만듭니다.